# Problem Set-up:

The monitoring network optimization problem has the follwing main components: 

1. Compare streamflow probability distributions (compute pairwise KL divergence), 
2. Develop a model to predict divergence of distributions from basin attributes, and 
3. Apply the model to the ungauged basin (decision) space.
4. Express the uncertainty in the expected information gain from (greedy) network expansion
5. Rank the existing network for expected total information loss from network contraction

## Make a large number of probability distribution comparisons 

* the observed data $X = \{(R_1, B_1), (R_2, B_2), \dots, (R_N, B_N)\}$ represents a network of $\text{M}$ streamflow monitoring locations, where:  
  * $R$ is a vector of **mean daily unit area runoff (UAR)** over a period of observation, and 
  * $B$ is a vector of **basin attributes** (assumed to be static in time)
* given $R$, let $Z = C(R)$, where $C \in \mathcal{C}$ is a quantization (encoding) scheme among many possible.  
    * **Problem 1**: How can $C$ be chosen to minimize quantization noise and maximize the mutual information between R and Z?
* let $\mathcal{R} = R\times R$ represent the set of all **pairs** of $R$ (the Cartesian product), denoted as $\{ (R_i, R_j) | R_i, R_j \in R  \text{ and } i \neq j \}$ 
    * $(R_i, R_j) \in \mathcal{R}$ is subject to a minimum overlapping interval ($t_{min}$) between $R_i$ and $R_j$.
* let $Y$ denote the input variable (the covariate vector) representing a measure of divergence between each $(R_i, R_j) \in \mathcal{R} \text{ where } i \neq j$ and the concurrent record criterion $t_{min}$ is met.
* $Y$ is then defined by the Kullback-Leibler (KL) divergence of an observed distribution $P$ and an approximate model of the observed data $Q$:  $$D_{KL}(P_i || Q_j) = \sum_{\omega \in \mathcal{\Omega}} p_i(\omega) \log \left( \frac{p_i(\omega)}{q_j(\omega)} \right) $$
    * $P$ is the probability mass function (PMF) describing the observed distribution of UAR ($R_i$), 
    * $Q$ is the PMF of "simulated" UAR obtained by assuming UAR is equal to $R$ at location $j$.  This is simply an equal unit area runoff model. 
    * $P$ and $Q$ are defined for all states $\omega \in \Omega$, and $\Omega$ is defined by the quantization scheme $C$.
* **Problem 2**: $q(\omega_i) = 0$ is inadmissible, how can a prior distribution be set to have minimal influence?


## Predict divergence of distributions from basin attributes

* let $\delta (\theta)$ represent a gradient-boosted regression model (XGBoost) for predicting KL divergence $(Y)$ from basin attributes $(B)$, 
* let $\theta \in \Theta$ represent a model parameterization in the space of hyperparameters for $\delta$.

### Loss Function $\mathcal{L}(Y, \delta(\theta))$

* absolute error is used as the objective function in XGBoost because the residuals are not normally distributed and $\theta$ is sensitive to outliers.
* since $Y$ is generated from all **pairs** of $R$, leaving out $(R_i, R_j)$ pairs for validation is not enough since $R_i$ can occur in up to $M-1$ pairings (though in practice much smaller). 
* use Monte Carlo (MC) simulation to mitigate selection bias. 
    * $\theta$ is held constant across $n_{trials}$ **trials** in a given simulation.
    * let $S$ be a subset of $\mathcal{R}$ obtained by excluding $n_{exclude}$ elements from $R$.  Elements are excluded by:
        * random selection to test selection bias, and 
        * geographic location to test spatial bias,
    * the **mean loss** of one simulation (ensemble) is computed from the complement of $S$, $S^c = \{ (R_i, R_j) | (R_i, R_j) \notin S \}$ such that the model is tested out of sample $n_{trials}$ times.
* the parameter space $\Theta$ is searched repeatedly to find an approximately optimal parameterization $\theta^*$ that yields the lowest mean MC simulation loss from $\mathcal{L}(Y, \delta(\theta^*))$.
* **Problem 3**: how can model uncertainty be expressed?


## Apply the Model to the Decision Space

* let $A$ represent the space of ($k$) unmonitored basins. 
* given $\delta(\theta^*)$, let $\hat Y_{bk}$ represent the **baseline** information about $A_k$, defined as: 
    * the expected additional bits of information (per observation) needed to fully describe the unit area runoff at an unmonitored location $A_k$.
    * the baseline can also be thought of as the compression provided by the current network about the unmonitored basin $A_k$
* for each unmonitored basin $A_k \in A$, compute the baseline information: 
    * let $\hat Y_{bk}$ represent the minimum expected divergence among **monitored** basins $\{ (A_k, R_j) | R_i \in R \}$, given by:  
  $$\hat Y_{bk} = \min_{i : (A_k, R_i) \in A \times R} \delta(\theta^*)$$  
    * let $\hat Y_{kl}$ represent the minimum expected divergence among **unmonitored** basins $A_l \in A, k \neq l$. 
        * for each $A_k$, find the set of locations $A_l$ where the expected divergence is less than the baseline value for that location $\hat Y_{kl} < \hat Y_{kb}$. 
        * Where $\hat Y_{kl} < \hat Y_{kb}$ is satisfied, monitoring location $l$ is expected to provide more information about location $k$ than the current monitoring network provides.  
        * Monitoring at location $l$ is expected to reduce the surprise about $k$ by $\hat Y_{kb} - \hat Y_{kl}$ bits per sample, 
        * In other words reduce the number of yes/no questions needed to fully describe $R_k$
* let $G_k$ denote the total "surprise reduction" of $A_k$, defined as the expected information about the unmonitored space $A$ gained from monitoring element $A_k$.
* important to express uncertainty in $G_k$ as stated above re: **Problem 3**
* $G_k$ can be used to decide the next monitoring location (greedy selection), where greater values represent a greater expected amount of information about $A$


## Problems

### **Problem 1**: Quantization $(C)$

* $C$ is defined by the set of equal intervals $I$ that span from $R_{ij}^a = min\left(min(R_i), min(R_j) \right)$ to $R_{ij}^b = max\left(max(R_i), max(R_j)\right)$.
    * interval $I_k = \left[ R_{ij}^a + \frac{k(r_{ij}^b)}{n}, R_{ij}^a +  \frac{k+1(r_{ij}^b)}{n} \right]$ where $n = 2^b$ is the dictionary size
* it is advantageous to quantize runoff to address (some part of) heteroscedastic measurement error:  
    * log-transform runoff   
    * equal width binning (in log space) to reflect measurement error
* parametric vs. nonparametric distributions
    * parametric:
        * avoids $q(\omega) = 0$
        * information is lost in parametric fit
        * i.e. gamma works often, but what about bimodal distributions or ephemeral streams?
    * nonparametric:
        * must set prior to address $q(\omega) = 0$
        * information lost / noise added in quantization (4 vs. 6 vs. 8 bits) 
* $D_{KL}(P||Q)$ "seeks the mean" since:
    * $\sum p(\omega) = \sum q(\omega) = 1$
    * $D_{KL}$ penalty is proportional to $p(\omega)$
    * penalty is sensitive to small Q
    * no (direct) penalty assigned where $p(\omega) = 0, q(\omega) > 0$
        * indirectly penalized because $\sum q(\omega) < 1$ where $p(\omega) > 0$
    * $D_{KL} > 0$ is sub-optimal encoding, or "misallocation of bit-length" compared to optimal (observed / true distribution) due to incorrect frequency estimation
* we want to maximize mutual information between $R$ and $Z$
    * let $n = 2^b$ represent the dictionary size, where $b$ is the bitrate
    * larger dictionary preserves more information for $\delta$ to learn from
    * 6 bit quantization yields lower model error than 4 bit, 8 bits lower than 6, etc.
    * as dictionary size grows, overquantization starts to cause problems in $D_{KL}$ (empty bins)
    * $p(\omega) = 0$ over-quantization isn't a problem, but $Q = 0$ is prior distribution problem, discussed next.
* calculate $D_{KL}$ for increasing bitrate until over-quantization causes divergence
    * balances preserving information and suppressing quantization noise



### **Problem 2:** Choosing an Appropriate Prior Distribution

Given the divergence of the observed distribution $P$ from the simulated distribution $Q$ from the KL divergence:

$$D_{KL}(P_i || Q_j) = \sum_{\omega \in \mathcal{\Omega}} P_i(\omega) \log \left( \frac{p_i(\omega)}{q_j(\omega)} \right) $$

#### Unobserved state(s) $p(\omega) = 0$

For $p(\omega) = 0, q(\omega) > 0$, we assign $D_{KL}|_{p(\omega)=0} = 0$ by L'Hôpital's Rule:

$$\lim_{p(\omega) \to 0} \log \left(\frac{p(\omega_i)}{q(\omega_i)} \right) = \lim_{x \to 0^+} \frac{\log (x)}{x^{-1}}$$

* Differentiate $f(x) = \log(x)$ and $g(x) = x^{-1}$ with respect to x:
    * $f'(x) = \frac{d}{dx}\left[log(x) \right] = \frac{1}{x}$  
    * $g'(x) = \frac{d}{dx}\left[x^{-1} \right] = -x^{-2}$  
* Take the limit of the ratio of $f'(x)$ and $g'{x)}$ as $x \to 0$:  
    
$$\lim_{x \to 0} \frac{\frac{1}{x}}{\frac{-1}{x^2}} = \lim_{x \to 0} -x = 0$$



#### Unpredicted state(s) $q(\omega) = 0$

States where $q(\omega) = 0$ cannot be neglected since the resulting $D_{KL}$ would suggest falsely close distributions, in some cases yielding negative divergence.

* let $\alpha$ represent a prior distribution for the model predictions $Q$
* small $\alpha$ corresponds to strong belief in the model, and vice versa
* choose $\alpha$ that is minimally influential on the posterior $Q$
* observed records are of different lengths,
    * express $q(\omega)$ in terms of **event counts**
    * assume a uniform distribution of 1 pseudo-count $\alpha = (1)_{i=1}^{2^b}$
    * renormalizing by total counts implicitly adjusts the "strength of belief" in the observations.
        * longer records divide the prior (1) by a larger number resulting in a smaller prior, and vice-versa.


#### Varying $\alpha$

* consider the case where $0 > \alpha < 1$:
    * smaller $\alpha$ results in larger $D_{KL}$ where $q(\omega) = 0, p(\omega) > 0$
    * smaller $\alpha$ indicates increasing preference for selecting a model (proxy location) that prioritizes optimal encoding on **extreme frequencies**.
        * analogous to noise shaping in audio processing?
    * less clear implications for information / compression interpretation
* consider the case where $\alpha \gg 1$
    * increasing $\alpha$ corresponds to decreasing strength of belief in the model
        * could it also reflect weaker belief in the accuracy of observations (since Q is observed data at $j$)?
    * let $\mathcal{U}$ represent the uniform distribution in the range $[a, b]$.
    * $\lim_{\alpha \to \infty} D_{KL} = \sum p(\omega)\log \frac{p(\omega)}{\mathcal{U}}$
    * $\sum q(\omega)|_{p(\omega) = 0} < 1$, and it decreases by $1/N$ as $\alpha \to \infty$ for each state $p(\omega) = 0$
        * $D_{KL}$ is sensitive to small $p$, where a large $\alpha$ and small $p(\omega)$ yield $\frac{p(\omega)}{q(\omega)} < 1 \to \log \frac{p(\omega)}{q(\omega)} < 0$
        * as the number of unobserved states increases (overquantization), $D_{KL}$ increases since the ratios $\frac{p(\omega)}{q(\omega)}$ increase with decreasing $q(\omega)$
* For $(R_i, R_j)$ pairs where $q(\omega) > 0 \text{ for all } \omega$, assuming a uniform $\alpha = (1)_{i=1}^{2^b}$ yields very little influence on $Q$ since the minimum number of observations is $\approx 10^3$,
* Where $q(\omega) = 0, p(\omega) > 0$, $D_{KL}$ is minimized when the prior $\alpha = (1)_{i=1}^{2^b}$ is assumed, and it increases as $\alpha$ changes in both directions towards zero and infinity.


### **Problem 3**: Model Uncertainty

* combine all predicted and observed values from the MC simulation trials
* define some number of intervals ($n_{intervals}$) of $\hat Y$ and compute the distribution of $Y$ in each interval. 
    * set intervals (bins) by:
        * minimum sample size
        * minimum "interesting" difference in $\hat Y$
    * because we've used Monte Carlo simulation to generate large samples in each bin, we can make a two-sided t-test arbitrarily sensitive to differences in mean between bins, so p-values are kind of meaningless and we should incoporate an effect size.
* The final step is to map all $G_{k} > 0$ back to spatial coordinates to serve as a decision tool, 
    * greater $G_{k}$ is associated with greater expected information for the total decision space by adding a monitoring station at location $k$.
    * how should uncertainty in $G_k$ be expressed given:
        * model residuals are heteroscedastic
            * can we assume errors are independent?
        * each $G_k$ is the sum of many predicted values from as many as $n_{intervals}$ unique error distributions
* The aim is to provide confidence intervals for $G_k$ to use in comparing the expected information gain from monitoring one location over another
    * let $\epsilon$ represent the minimum expected additional information in order to consider monitoring at one location to be advantageous over another.
        * $G_k - G_l > \epsilon$ can then be used to reduce the complexity / computation / decision space

### Network Contraction Problem

Since network contraction doesn't eliminate existing information:  

* how do we value continuous/current/historical records?
* how is marginal information valued?
* is there an optimal replacement strategy path (i.e. move n stations)?
* i.e. incorporate factors describing:
    * reliability (hydraulic control stability, site exposure)
    * servicing cost (travel time, remoteness factor)
    * worker safety factor (maybe same as remoteness)
    * cost of aquiring existing private data in vicinity

.


.




.



.




.



.




.



.









## Random Forest Vs. Gradient-Boosted Trees

### RF
* build a large number of *complete* decision trees based on bootstrap resampling, average the predictions.
* each tree is trained on a bootstrap (random) sample and builds trees by random feature subsets at each node (supposed to help protect against overfitting)

### GBDT
* build (ensembles of small) trees in series, weightings in subsequent trees are adjusted by errors of previous iteration.
* uses gradient descent to add new model




Given a dataset $X$ consisting of $N$ observations, where each observation $x_i$ comprises two components: a time series $T_i$ and static descriptive indices $D_i$, we aim to find the model parameters $\theta$ that minimize the discrepancy between observed outcomes $Y$ and predicted outcomes $\hat{Y}$. The discrepancy is quantified by the loss function $\mathcal{L}(Y, \hat{Y})$.

- **Data Representation**: 
  - $X = \{(T_1, D_1), (T_2, D_2), ..., (T_N, D_N)\}$

- **Quantization Scheme**: 
  - $Z_i = Q(T_i)$, where $Q$ is a quantization scheme applied to the time series component $T_i$ to produce a discretized representation $Z_i$.

- **Model**: 
  - $\hat{Y}_i = \delta(D_i; \theta)$, where $\delta$ is the model operating on the static descriptive indices $D_i$ parameterized by $\theta$, to predict outcomes $\hat{Y}_i$.

- **Loss Function**: 
  - $\mathcal{L}(Y, \hat{Y})$ quantifies the discrepancy between observed outcomes $Y$ and predicted outcomes $\hat{Y}$.

The objective is to find the optimal model parameters $\theta$ that minimize the loss function $\mathcal{L}(Y, \hat{Y})$.

## General Form

Given a large dataset $X$ of dimension $M$, define and calibrate a model $\delta(\theta)$ to minimize the discrepancy between observed outcomes $Y$ and predicted outcomes $\hat Y$.  The discrepancy is quantified by a loss function $l(Y, \hat Y)$ where $\hat Y$ is obtained through the model $\delta (Z;\theta)$.  Here, $Z$ is a transformed representation of X achieved through a quantization strategy $Q$, such that $Z = Q(U)$.  The quantization strategy $Q$ is chosen to minimize quantization noise and maximize the mutual information between X and Z.

## Context

The first step is to find the quantization mapping $C$ to $Z$, $Z=Q(U)$, that maximizes the entropy of $Y(Z)$: $$\max\limits_{Q} H(Y(Q(U))$$

under the condition that $Q$ is penalized by the quantization noise $R(Q)$: $$\max\limits_{Q} H(Y(Q(U))) - \lambda R(Q)$$

The second step is to minimize the prediction error (or loss $l$) of model $\delta(Z;\theta)$: $$\min\limits_{\delta,\theta} l(Y, \delta(Z; \theta)$$

The third step is to be able to map new $X$ to $\hat Y$ and express risk derived from Monte Carlo Simulation where $\delta$ is minimized by $\theta^*$ for random samples of $X^* \in \hat X$.

### Model Results Exploration

Using Monte Carlo simulation, we have generated a large sample of $\theta$ to evaluate via $\delta(\theta)$ using xgboost, in other words we have a distribution of $\theta$ for a given $\delta$.  

The goal now is to evaluate the loss as a function of the predicted $\hat Y$.  To do this, we set up some number of simulations where we hold $\theta$ fixed but randomly select K stations to leave out of the training data completely (since each training sample is a pair of basins, the K stations must be left out of pairings altogether.  



In [ ]:
import os
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as st

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, column, row
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20, Set2
import numpy as np
output_notebook()

BASE_DIR = os.path.dirname(os.getcwd())

We used gradient boosted decision trees to find a model that tries to predict divergence of runoff distributions from basin attributes.

Along with $D_{KL}$ measures as a function of quantization $C$ and prior $\alpha$ are other common measures: 
* coefficient of determination ($R^2$) 
* Nash Sutcliffe Efficiency (NSE)
* Kling-Gupta Efficiency (KGE), and
* Total Variation distance

Before getting into XGBoost model calibration, let's look at the distribution of the above metrics across quantization schemes.

In [ ]:
def load_data(b, revision_date='20240409'):
    result_folder = os.path.join(BASE_DIR, 'processed_data', 'dkl_test_results')
    result_fname = f'DKL_results_{b}bits_{revision_date}.csv'
    results_fpath = os.path.join(result_folder, result_fname)
    df = pd.read_csv(results_fpath, dtype={'proxy': str, 'target': str})
    return df

In [ ]:
bitrates = [4, 5, 6, 7, 8]
bitrates = [8]

model_cols = ['cod', 'nse', 'kge', 'tvd'] 
dkl_cols = ['dkl_sim', 'dkl_post_0.001R', 'dkl_post_0.01R', 'dkl_post_0.1R',
       'dkl_post_0R', 'dkl_post_1R', 'dkl_post_2R', 'dkl_post_5R',
       'dkl_post_10R', 'dkl_post_20R', 'dkl_post_50R', 'dkl_post_100R',
       'dkl_post_200R', 'dkl_post_500R', 'dkl_post_1000R']

In [ ]:
# get all data in the same dataframe
rdf = pd.DataFrame()

for b in bitrates:
    df = load_data(b)
    pairs = df.apply(lambda row: f"{row['proxy']}-{row['target']}", axis=1).values
    df['id'] = pairs
    df.set_index('id', inplace=True)
    data = df[dkl_cols + model_cols].copy()
    data.columns = [f'{b}_{c}' for c in data.columns]
    if rdf.empty:
        rdf = data.copy()
    else:
        rdf = pd.concat([rdf, data], join='inner', axis=1)


In [ ]:
plots = []
for c in model_cols:
    fig = figure(title=c, x_axis_label=c, y_axis_label='P(X)')
    n = 0
    for b in bitrates:
        vals = rdf[f'{b}_{c}'].values
        hist, edges = np.histogram(vals, bins=50, density=True)
        x = (edges[:-1] + edges[1:]) / 2
        fig.line(x, hist, line_color=Set2[6][n], legend_label=f'{b}bits',
                line_width=3)
        n += 1
    plots.append(fig)
        

In [ ]:
layout = gridplot(plots, ncols=2, width=600, height=350)
show(layout)

In [ ]:
plots = []
for c in dkl_cols:
    fig = figure(title=c, x_axis_label=c, y_axis_label='P(X)')
    n = 0
    for b in bitrates:
        vals = rdf[f'{b}_{c}'].values
        hist, edges = np.histogram(vals, bins=50, density=True)
        x = (edges[:-1] + edges[1:]) / 2
        fig.line(x, hist, line_color=Set2[6][n], legend_label=f'{b}bits',
                line_width=3)
        n += 1
    plots.append(fig)

In [ ]:
layout = gridplot(plots, ncols=2, width=600, height=350)
show(layout)

## Next Steps

**At the individual pair level** check how much the $D_{KL}$ measure diverges for all $\alpha$ as we go from $4 \to 8$ bits encoding.  What is the effect of underquantization occurring in the middle instead of at the edges?  It seems to be a vestige of the quantization and the metric $D_{KL}$, but it still represents a real inefficiency in encoding.

How can the appearance of "missing bins" be interpreted.  The greater the detail we look at distributions,  to match as we increase the dictionary size.  The penalty is implicitly adjusted here as well since the uniform prior distributes $\frac{\alpha_i}{2^b}$ probability to each bin, so as the dictionary size increases, the penalty 

We do not apply a prior to $P$.

In [ ]:
ddf = pd.DataFrame()
for c in dkl_cols:
    if c.endswith('dkl_sim'):
        continue
    # for b in bitrates:
    # for bi in range(1, len(bitrates)):
    baseline_col = f'4_{c}'
    for b in bitrates[1:]:
        ddf[f'{b}_{c}_diff'] = rdf[f'{b}_{c}'] - rdf[baseline_col]
    

In [ ]:
plots = []
print(dkl_cols)
for c in [e for e in dkl_cols if not e.endswith('_0R')]:
    if c == 'dkl_sim':
        continue
    fig = figure(title=f'quantization assessment: {c}', width=800, height=400)
    n = 0
    for b in bitrates[1:]:
        vals = ddf[f'{b}_{c}_diff'].values
        hist, edges = np.histogram(vals, bins=50, density=True)
        x = (edges[:-1] + edges[1:]) / 2
        fig.line(x, hist, line_color=Set2[len(bitrates)][n], legend_label=f'{b}bits',
                line_width=3)
        n += 1
    plots.append(fig)


In [ ]:
layout = gridplot(plots, ncols=5, width=300, height=200)
show(layout)

In [ ]:
plots  = []
for c in [e for e in dkl_cols if not e.endswith('_0R')]:
    if c == 'dkl_sim':
        continue
    if c.endswith('_1R'):
        continue
    fig = figure(title=f'{c}', width=800, height=400, output_backend='webgl')
    n = 0
    baseline_vals = rdf[f'4_{c}'].copy()
    for b in bitrates[1:][::-1]:
        vals = rdf[f'{b}_{c}'].values
        fig.circle(baseline_vals, vals, color=Set2[len(bitrates)][n], legend_label=f'{b}bits',
                size=1, alpha=0.2)
        n += 1
    fig.legend.click_policy='hide'
    plots.append(fig)

In [ ]:
layout = gridplot(plots, ncols=5, width=300, height=200)
show(layout)

### Aggregate results from trials across an MC simulation

In [ ]:

# get the files for one simulation
# sim_identifier = 'n_estimators_75'
# max_depth_id = 'max_depth_7'
# lr_id = 0.144
# sim_identifier = 'n_estimators_98'
# max_depth_id = 'max_depth_6'
# lr_id = 0.07
# sim_identifier = 'n_estimators_93'
# max_depth_id = 'max_depth_6'
# lr_id = 0.112
# estimators = 35
# max_depth = 7
# lr_id = 0.234
# estimators = 33
# max_depth = 8
# lr_id = 0.196
# estimators = 97
# max_depth = 8
# lr_id = 0.141
# estimators = 97
# max_depth = 8
# lr_id = 0.310
estimators = 89
max_depth = 5
lr_id = 0.093

sim_identifier = f'n_estimators_{estimators}'
max_depth_id = f'max_depth_{max_depth}'


sim_files = sorted([e for e in os.listdir('xval_results') if sim_identifier in e])
sim_files = [e for e in sim_files if max_depth_id in e]
sim_files = [e for e in sim_files if round(float(e.split('_')[16]), 3) == lr_id]
# sim_files = sorted([e for e in os.listdir('xval_results') if int(e.split('_')[0]) == sim_id])
print(len(sim_files))
print(sim_files[0])
# for i, v in enumerate(sim_files[0].split('_')):
#     print(i, v)

In [ ]:
sim_df = pd.DataFrame()
sim_df['file'] = sim_files
sim_df["sim_no"] = sim_df["file"].apply(lambda x: int(x.split("_")[0]))
sim_df["bits"] = sim_df["file"].apply(lambda x: int(x.split("_")[2]))
sim_df["max_depth"] = sim_df["file"].apply(lambda x: int(x.split("_")[10]))
sim_df['prior'] = sim_df['file'].apply(lambda x: float(x.split('_')[4]))
sim_df["n_estimators"] = sim_df["file"].apply(lambda x: int(x.split("_")[13]))
sim_df["learning_rate"] = sim_df["file"].apply(lambda x: float(x.split("_")[16]))
sim_df["colsample_bytree"] = sim_df["file"].apply(lambda x: float(x.split("_")[19]))
# sim_df["lambda"] = sim_df["file"].apply(lambda x: float(x.split("_")[19]))
# sim_df["alpha"] = sim_df["file"].apply(lambda x: float(x.split("_")[21]))
sim_df["mse"] = sim_df["file"].apply(lambda x: float(x.split("_")[21]))
sim_df["n_train"] = sim_df["file"].apply(lambda x: int(x.split("_")[22]))
sim_df["n_test"] = sim_df["file"].apply(lambda x: int(x.split("_")[24]))
sim_df.sort_values('sim_no', inplace=True)
sim_df.head()

In [ ]:
bits = list(set(sim_df['bits']))[0]
prior = list(set(sim_df['prior']))[0]

First, lets get an understanding of the distribution of predictions.

In [ ]:
# first, lets get an understanding of the distribution of predicted values
predictions = []
actuals = []
eqp_edges = []
eqw_edges = []
n_bins = 20
all_max, all_min = -1e6, 1e6
for i, row in sim_df.iterrows():
    sim_id = row['sim_no']
    file = row['file']
    mae = row['mse']
    sim = pd.read_csv(f'xval_results/{file}')

    preds = sim['predicted'].values
    predictions += list(preds)
    actuals += list(sim['actual'].values)
    if min(preds) < all_min:
        all_min = min(preds)
    if max(preds) > all_max:
        all_max = max(preds)
    # add a small amount of random noise to avoid ties
    preds += np.random.uniform(-1e-6, 1e-6, len(preds))
    sim_sorted = sorted(preds)
    hist, edges = np.histogram(sim_sorted, n_bins-1, density=True)
    eqw_edges.append(list([0] + edges.tolist()))
    
    # Calculate the percentiles to split the preds array
    percentiles = np.linspace(0, 100, n_bins + 1)
    
    # Compute the bin edges using the percentiles
    bin_edges = np.percentile(preds, percentiles)
    
    # Ensure uniqueness by handling potential duplicates in bin_edges
    unique_bin_edges = np.unique(bin_edges)
    # Handle the rare case where uniqueness filtering reduces the number of edges
    if len(unique_bin_edges) < len(bin_edges):
        raise ValueError("Reduced bin edges due to duplicates. Consider fewer bins or different data.")
    else:
        eqp_edges.append(unique_bin_edges)
    

print('min: ', all_min, ' max: ', all_max)

In [ ]:
adf = pd.DataFrame({'Predicted': predictions, 'Observed': actuals})
# Compute the 95% confidence interval
min_pred, max_pred = adf['Predicted'].min(), adf['Observed'].max()
eval_x = np.arange(min_pred, max_pred, 0.1)


In [ ]:
source = ColumnDataSource(adf)
afig = figure(width=600, height=400, title=f'Aggregated MC Simulation ({bits}bits, {prior}prior)', output_backend='webgl')
afig.circle('Predicted', 'Observed', color='dodgerblue', size=1, alpha=0.5, source=source)
# afig.line(eval_x, smoothed, line_width=3, line_color='black')
# afig.line(eval_x, bottom, line_width=3, line_color='black', line_dash='dotted')
# afig.line(eval_x, top, line_width=3, line_color='black', line_dash='dotted')
afig.line([0, max(adf['Predicted'])], [0, max(adf['Predicted'])], color='red', width=3, line_dash='dashed')
afig.xaxis.axis_label = 'Predicted D_KL'
afig.yaxis.axis_label = 'Actual D_KL'
# show(afig)

In [ ]:
ed_df = pd.DataFrame()
ed_df['ew'] = pd.DataFrame(eqw_edges).mean(0)
ed_df['ep'] = pd.DataFrame(eqp_edges).mean(0)
ed_df.loc[0, :] = all_min - 1e-6
ed_df.loc[max(ed_df.index), :] = all_max + 1e-6
# ed_df

In [ ]:
ew_counts, ep_counts = [], []
ep_probs, ew_probs = [], []
for i, row in sim_df.iterrows():
    sim_id = row['sim_no']
    file = row['file']
    mae = row['mse']
    sim = pd.read_csv(f'xval_results/{file}')
    preds = sim['predicted'].values
    # ew_bin_counts = np.zeros(n_bins)
    # ep_bin_counts = np.zeros(n_bins)
    ew_edges = ed_df['ew'].to_numpy()
    ep_edges = ed_df['ep'].to_numpy()
    
    ew_bins = np.digitize(preds, ew_edges) - 1
    ep_bins = np.digitize(preds, ep_edges) - 1
    # unique_bins, counts = np.unique(ew_bins, return_counts=True)
    
    ew_bin_counts = np.bincount(ew_bins, minlength=n_bins)
    ep_bin_counts = np.bincount(ep_bins, minlength=n_bins)
        
    # Calculate probabilities
    ew_prob = ew_bin_counts / ew_bin_counts.sum()
    ep_prob = ep_bin_counts / ep_bin_counts.sum()
    
    ew_probs.append(ew_prob.tolist())
    ep_probs.append(ep_prob.tolist())


In [ ]:
pdf = pd.DataFrame()
pdf['ew'] = pd.DataFrame(ew_probs).mean(0)
pdf['ep'] = pd.DataFrame(ep_probs).mean(0)
# pdf

In [ ]:
ew_widths = np.diff(ed_df['ew'])
ep_widths = np.diff(ed_df['ep'])
ew_cents = ed_df['ew'][:-1] + ew_widths / 2
ep_cents = ed_df['ep'][:-1] + ep_widths / 2

source_df = pd.DataFrame()
source_df['ep_cents'] = ep_cents
source_df['ew_cents'] = ew_cents
source_df['ep_p'] = pdf['ep'].values
source_df['ew_p'] = pdf['ew'].values
source_df['ew_w'] = ew_widths
source_df['ep_w'] = ep_widths
print(len(source_df))
print(len(ed_df))
source = ColumnDataSource(source_df)



In [ ]:
# Create a Bokeh plot (vertical bars)
p = figure(width=600, height=400, title="Histogram of Probabilities",
           x_axis_label='Bins', y_axis_label='Probability')

# Add vertical bars to the plot
p.vbar(x='ew_cents', top='ew_p', width='ew_w', source=source, fill_alpha=0.6,
       legend_label="Equal Width", fill_color='navy', line_color='white')
p.vbar(x='ep_cents', top='ep_p', width='ep_w', source=source, fill_alpha=0.6,
       legend_label="Equal Probability", fill_color='orange', line_color='white')

p.legend.click_policy='hide'
# show(p)


For both binning methds, iterate through the simulations and collect the containing residuals.

The bins will get very large, so from these we can compute in-bin distributions to evaluate the precision.

In [ ]:
def compute_percentiles(df, column_name, bin_size):
    """
    Compute the 5th, 50th, and 95th percentiles for every bin_size chunk of ordered rows 
    in a DataFrame based on the specified column.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.
    column_name (str): The name of the column to compute percentiles for.
    bin_size (int): The number of rows in each chunk.

    Returns:
    pandas.DataFrame: A DataFrame containing the percentiles for each chunk.
    """
    # Initialize a list to store the percentile results
    percentiles_list = []
    final_bin = False
    for start_idx in range(0, len(df), bin_size):
        
        end_idx = start_idx + bin_size
        
        if len(df) - end_idx < bin_size:
            # Adjust the end index of the previous chunk to include the last bit
            end_idx = len(df)
            final_bin = True
        chunk = df.iloc[start_idx:end_idx]
        
        # Calculate the percentiles for the current chunk
        percentiles = np.percentile(chunk[column_name], [5, 50, 95])
        mean_predicted = chunk['predicted'].mean()
        
        # Append the results to the list
        percentiles_list.append({
            'start_index': start_idx,
            'end_index': end_idx,
            'mean_predicted': mean_predicted,
            '5': percentiles[0],
            '50': percentiles[1],  # Median
            '95': percentiles[2],
        })
        if final_bin:
            break
    
    # Convert the list of dictionaries to a DataFrame and return
    return pd.DataFrame(percentiles_list)

In [ ]:
data = {}
bin_vals = {}
label_dict = {}
pred_vals, obs_vals = [], []
for method in ['ep', 'ew']:
    method_results = []
    bin_vals[method] = {}
    label_dict[method] = {}
    for i, row in sim_df.iterrows():
        sim_id = row['sim_no']
        file = row['file']
        mae = row['mse']
        sim = pd.read_csv(f'xval_results/{file}')
        sim.drop('Unnamed: 0', axis=1, inplace=True)
        # preds = sim['predicted'].values
        # sort the dataframe by the predicted value
        sim.sort_values('predicted', inplace=True)
        sim.reset_index(inplace=True, drop=True)
        pred_vals += list(sim['predicted'].values)
        obs_vals += list(sim['actual'].values)
        

In [ ]:
# sort the aggregated data by predicted value and compute the confidence interval for every 1000 pts
adf = pd.DataFrame({'predicted': pred_vals, 'actual': obs_vals})
adf.sort_values('predicted', inplace=True)
ci_df = compute_percentiles(adf, 'actual', 10000)
# ci_df

In [ ]:

ew_widths = np.diff(ed_df['ew'])
ew_cents = ed_df['ew'][:-1] + ew_widths / 2
hs_df = pd.DataFrame()
hs_df['ew_cents'] = ew_cents
hs_df['ew_p'] = pdf['ew'].values
hs_df['ew_w'] = ew_widths

In [ ]:
source = ColumnDataSource(adf)
afig = figure(width=600, height=400, title=f'Aggregated MC Simulations ({estimators} estimators {max_depth} max depth)', output_backend='webgl')
afig.circle('predicted', 'actual', color='dodgerblue', size=0.5, alpha=0.25, source= source)
afig.line(ci_df['mean_predicted'], ci_df['50'], line_width=3, line_color='black')
afig.line(ci_df['mean_predicted'], ci_df['5'], line_width=3, line_color='black', line_dash='dotted')
afig.line(ci_df['mean_predicted'], ci_df['95'], line_width=3, line_color='black', line_dash='dotted')
afig.line([0, max(adf['predicted'])], [0, max(adf['predicted'])], color='red', width=3, line_dash='dashed')
# afig.xaxis.axis_label = r'Predicted $$D_{KL}$$'
# afig.yaxis.axis_label = r'Actual $$D_{KL}$$'
afig.xaxis.axis_label = 'Predicted DKL'
afig.yaxis.axis_label = 'Actual DKL'



source = ColumnDataSource(hs_df)
h = figure(width=600, height=200, title="Distribution of Predicted Values",
           x_axis_label=r'$$\hat Y$$', y_axis_label='Probability')
h.x_range = afig.x_range

# Add vertical bars to the plot
h.vbar(x='ew_cents', top='ew_p', width='ew_w', source=source, fill_alpha=0.6,
       legend_label="Equal Width", fill_color='navy', line_color='white')

h.legend.click_policy='hide'

layout = column(afig, h)
show(layout)

In [ ]:
def compare_bins_with_indices(main_df, bin_df, observed_col, predicted_col, alpha=0.05):
    """
    Performs pairwise comparisons between bins of values using the Mann-Whitney U test,
    based on the main dataframe of observed and predicted values, and a dataframe of bin start indices.
    
    Parameters:
    main_df (pandas.DataFrame): DataFrame containing observed and predicted values.
    bin_df (pandas.DataFrame): DataFrame containing the bin start indices.
    observed_col (str): Column name for observed values in main_df.
    predicted_col (str): Column name for predicted values in main_df.
    alpha (float): Significance level for determining statistical difference.
    
    Returns:
    pd.DataFrame: A matrix (DataFrame) of p-values from the Mann-Whitney U tests.
    """
    df = main_df.copy()
    bdf = bin_df.copy()
    # Add end index to bin_df for easier slicing
    bdf['end_index'] = bdf['end_index'].astype(int)
    bdf['start_index'] = bdf['start_index'].astype(int)
    
    # Sorting main_df by predicted values if not already sorted
    df.sort_values(by=predicted_col, inplace=True)
    df.reset_index(inplace=True)
        
    num_bins = len(bdf)
    p_values_matrix = pd.DataFrame(np.nan, index=range(num_bins), columns=range(num_bins))
    
    for i in range(num_bins):
        if i % 10 == 0:
            print(f'Processing {i}/{num_bins}')
        start_i, end_i = bdf.loc[i, ['start_index', 'end_index']].values
        start_i, end_i = int(start_i), int(end_i)
        bin_i = df.loc[start_i: end_i, observed_col].copy()

        p_value = np.nan
        if bin_i.empty:
            print(f'bin {i} has 0 len')
            continue
        
        for j in range(i + 1, num_bins):
            start_j, end_j = bdf.loc[j, ['start_index', 'end_index']].values
            start_j, end_j = int(start_j), int(end_j)
            bin_j = df.iloc[start_j:end_j][observed_col].copy()
            
            if bin_j.empty:
                print(f'bin {j} has 0 len')
                continue
            
            stat, p_value = st.mannwhitneyu(bin_i, bin_j, alternative='two-sided')
            p_values_matrix.at[i, j] = p_value
            p_values_matrix.at[j, i] = p_value  # The matrix is symmetric
    
    # Adjusting p-values for multiple comparisons if necessary
    # Example: Bonferroni correction
    # Adjusted alpha = alpha / number of comparisons
    # num_comparisons = num_bins * (num_bins - 1) / 2
    # adjusted_alpha = alpha / num_comparisons
    
    # Identifying significantly different pairs
    significantly_different_pairs = (p_values_matrix < alpha).stack()
    print(f"Significantly different pairs of bins (at alpha level of {alpha:.3f}):")
    print(significantly_different_pairs[significantly_different_pairs].index.tolist())
    
    return p_values_matrix


In [ ]:
# compare_bins_with_indices(adf, ci_df, 'actual', 'predicted', alpha=0.00001)